## Goal

Introduction to scRNAseq analysis using scanpy

## Who am I?

- Irene Robles Rebollo
- BSc in Biochemistry
- MSc in Applied Biosciences and Biotechnology
- PhD in Cell Biology
    - Half experimental
    - Half computational
- Bioinformatician at Cosyne Therapeutics


## History

| Year | Achievement | 
|------|-------------|
| 1953 |  DNA structure                   
| 1985 |  First PCR -  DNA amplification 
| 1988 |  PCR with Taq polymerase - Automated DNA amplification| 
| 1993 |  First qPCRs - RNA amplification 
| 1995 |  Microarrays - Quantify an array gene expression using a chip| 
| 2001 |  First draft of the human genome
| 2003 |  First NGS DNA sequencer - High throughput DNA sequencing |
| 2006 |  First RNA-seq -  High throughput RNA sequencing |
| 2009 |  First single-cell RNA-seq [@tang2009mrna] 
| 2013 |  Single-cell RNA-seq is declared method of the year

[@zhu2020pcr; @gondane2023revealing]


## Single-cell vs bulk RNAseq

:::: {.columns}

::: {.column width="49%"}

![Bulk RNA-seq, <a href="https://unsplash.com/es/@lyfefuel?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">LyfeFuel</a> from <a href="https://unsplash.com/es/fotos/bebida-verde-y-blanca-con-pajita-en-vaso-transparente-_82CV9I-TP8?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>](figures/lyfefuel-_82CV9I-TP8-unsplash.jpg)
:::


  
::: {.column width="3%"}
<!-- empty column to create gap -->
:::

::: {.column width="48%"}
![scRNA-seq, <a href="https://unsplash.com/es/@vdphotography?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">VD Photography</a> from <a href="https://unsplash.com/es/fotos/un-plato-blanco-cubierto-con-fruta-cortada-sEK8OMsJJ08?utm_content=creditCopyText&utm_medium=referral&utm_source=unsplash">Unsplash</a>](figures/vd-photography-sEK8OMsJJ08-unsplash.jpg)
:::

::::


|Feature              | Bulk data                          | Single-cell data                  |
|-----------------------|------------------------------------|-----------------------------------|
| Cell resolution       | Average of all cells               | Individual cell resolution        |
| Sample representation | Vector of gene expression values | Matrix of gene expression values  |
| Genomic resolution  | Higher, depends on sequencing depth  | Lower, depends on starting material |
| Cost                | Lower                                | High                              |
| Computational requirements | Lower                        | Higher                              |
| Data size           | Lower                               | Higher                           |
| Data interpretation | Simple                             | Complex                           |


## Sample representation

- In bulk data, each sample is repressented by a vector, where each value is a gene.
- In single cell data, each sample is a matrix, where each row is a gene and each column is a cell.

\begin{align}
Bulk &= \begin{bmatrix} gene_{1} \\ gene_{2} \\ gene_{3}\\ \vdots \\ gene_{n} \end{bmatrix} 
\\
\\

Single-cell &= \begin{bmatrix}
    gene_1, cell_1 & gene_1, cell_2 & gene_1, cell_3 & \dots  & gene_1, cell_m \\
    gene_2, cell_1 & gene_2, cell_2 & gene_2, cell_3 & \dots  & gene_2, cell_m \\
    gene_3, cell_1 & gene_3, cell_2 & gene_3, cell_3 & \dots  & gene_3, cell_m \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\
    gene_n, cell_1 & gene_n, cell_2 & gene_n, cell_3 & \dots  & gene_n, cell_m
\end{bmatrix}

\end{align}



## Scanpy vs Seurat
- Scanpy is a Python package for single-cell analysis [@wolf2018scanpy]
- Seurat is an R package for single-cell analysis [@satija2015spatial]
- Both are: 
    - User-friendly tools for single-cell analysis
    - Open source 
    - Well-documented
    - Widely-used
- Choice depends on:
    - Language preference
    - Team expertise
    - Integration with downstream analysis
    - Speed and memory requirements

> Hint: A good bioinformatician is not restricted by language. You can use R in Python can be done using the `rpy2` package. And Python can be use within R using `reticulate`. 

## Scale of scRNAseq data

![Number of cells per study over years [@svensson2020curated]](number_of_cells.png)

## AnnData object

![AnnData object, source: scanpy web](https://falexwolf.de/img/scanpy/anndata.svg) 

## Set up

- Install Miniconda
- Create a new environment

```bash
conda create -n myscanpy python=3.10
conda activate myscanpy
pip install -r requirements.txt
```

## Import libraries


In [ ]:
import scanpy as sc
import scrublet as scr
import numpy as np
import os
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

Scanpy setttings

In [ ]:
sc.settings.verbosity = 3   # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

## Download data

```bash
mkdir data
cd data
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE190nnn/GSE190622/suppl/GSE190622%5Fcount%5Fmatrix%5FAnnotated.csv.gz
```

## Load tests datasets

Single-cell SMART-seq data from mouse macrophages [@robles2022cohesin]


In [ ]:
mf = pd.read_csv("data/GSE190622_count_matrix_Annotated.csv.gz", index_col=0)
adata = sc.AnnData(X=mf.T)
adata.obs["genotype"] = adata.obs_names.to_series().apply(lambda x: x.split("_")[0])
adata.obs["timepoint"] = adata.obs_names.to_series().apply(lambda x: x.split("_")[1])
adata.obs["sample"] = adata.obs["genotype"] + "_" + adata.obs["timepoint"]
adata.var_names_make_unique()
adata

## Doublets

- A doublet is a single-cell transcriptome that has been generated by two cells
- They have often have a higher read and gene count
- They can produce confounding results

![Doublets, adapted from [@wolock2019scrublet]](figures/doublets_scubletpaper.png)

How do we fight doublets:
- Experiment design
- Computationally

### Scrublet

Scrublet finds doublets based os simulations [@wolock2019scrublet]

![Scrublet algorithm overview [@wolock2019scrublet]](figures/scrublet.png)


In [ ]:
scrub = scr.Scrublet(adata.X, expected_doublet_rate=0.06)
doublet_scores, predicted_doublets = scrub.scrub_doublets()
scrub.plot_histogram()

Adjust threshold


In [ ]:
predicted_doublets = scrub.call_doublets(threshold=0.2)
scrub.plot_histogram()

Visualise doublets


In [ ]:
scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
scrub.plot_embedding('UMAP', order_points=True);

Filter out doublets


In [ ]:
adata.obs["scrublet_doublet"] = predicted_doublets
adata = adata[adata.obs["scrublet_doublet"].apply(lambda x: x is False)]

## Preprocessing

### Highest expressing genes
Look for suspects: MALAT1, mitochondrial genes, ribosomal genes, componenets of the cytoskeleton, etc.


In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

### Quality metrics

`sc.pp.calculate_qc_metrics` computes quality control metrics for each cell.
- Number of counts per cell
- Number of genes per cell
- Percentage of counts that come from mitochondrial genes.


In [ ]:
# Mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, rotation = 90)

Different sampes might require different thresholds


In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True, groupby='sample', rotation = 90)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color='sample')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color='sample')

In this case, cells from different samples are reasonably homogeneous. However, that is not always the cells_per_paper.ipynb

![Example case where different tissues might require different quality control threshols, fraction of Tabula Muris data [@schaum2018single]](figures/violinqc_different_tissues.png)


## Filtering

Filter cells based on quality metrics


In [ ]:
sc.pp.filter_cells(adata, min_genes=500)
sc.pp.filter_genes(adata, min_cells=5)
adata= adata[adata.obs.n_genes_by_counts < 5000, :]
adata= adata[adata.obs.pct_counts_mt < 15, :]

## Normalisation: size normalisation and log transformation

For simplicity, we will use the simplest method: library size normalisation and log transformation, but there are others
[@risso2018general, @hafemeister2019normalization]. 

Steps:

- Transcript length normalisation: 
    - Adjusts for differences in transcript length between genes
    - Divides the counts in each cell by the length of the transcript
    - Not necessary if you sequence a fixed region of the transcript ( 3' or 5' in 10X, our case)
- Library size normalisation: 
    - Adjusts for differences in sequencing depth between cells
    - Divides the counts in each cell by the total counts in that cell and multiplies by a scale factor (e.g. 10,000)


In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

- Log transformation:
    - Logarithm of the normalised counts
    - Makes the data more normally distributed


In [ ]:
sc.pp.log1p(adata)

## Highly variable genes

- Genes that show the most variation across cells
- Often the most informative genes


In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

## Dimensionality reduction

### Goal
- Reducing the number of input variables or features in a dataset.

### Why 

- Noise Reduction: Focus on the most variance-contributing features, thus enhancing the signal-to-noise ratio.
- Data Visualization: We can plot in 2 or 3 dimensions
- Aid clustering and clasification
- Computational efficiency

### Dimensionality reduction techniques

- Principal Component Analysis (PCA): PCA is often the first step in dimensionality reduction for single-cell data. It identifies the principal components that capture the most variance in the data, reducing its dimensionality while preserving its structure as much as possible.
- t-Distributed Stochastic Neighbor Embedding (t-SNE): Popular dimensionality reduction that foccusses on the preservation on local structures
- Uniform Manifold Approximation and Projection (UMAP): Popular dimensionality reduction that foccusses on the preservation on global structures
- Autoencoders: Deep learning-based autoencoders can learn to compress single-cell data into a lower-dimensional representation in an unsupervised manner, capturing nonlinear relationships in the data.

## Prepare data for dimensionality reduction

The `.raw` attribute freezes the state of the AnnData object for later use. We can recumerate it by calling `.raw.to_adata().`


In [ ]:
adata.raw = adata

Filter by highly variable genes


In [ ]:
adata= adata[:, adata.var.highly_variable]
adata

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. 

In [ ]:
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

Scale each gene to unit variance. Clip values exceeding standard deviation to 10.

In [ ]:
sc.pp.scale(adata, max_value=10)

## Principal component analysis (PCA)

PCA is often the first step in dimensionality reduction for single-cell data. It identifies the principal components that capture the most variance in the data, reducing its dimensionality while preserving its structure as much as possible.


In [ ]:
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color='sample')

Varinace explained by each component:


In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

## UMAP and t-SNE

First, we need to compute the neighbourhood graph:


In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

Compute UMAP:


In [ ]:
sc.tl.umap(adata)

Compute t-SNE:


In [ ]:
sc.tl.tsne(adata)

Plot UMAP: 

In [ ]:
sc.pl.umap(adata, color='sample', use_raw=False)

Plot t-SNE: 

In [ ]:
sc.pl.tsne(adata, color='sample', use_raw=False)

## Clustering the neighborhood graph
As with Seurat and many other frameworks, we recommend the Leiden graph-clustering method (community detection based on optimizing modularity) by Traag et al. (2018). Note that Leiden clustering directly clusters the neighborhood graph of cells, which we already computed in the previous section.


In [ ]:
sc.tl.leiden(adata)
sc.pl.umap(adata, color='leiden', use_raw=False)

## Finding marker genes

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the .raw attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.


In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)

During the course of this analysis, the AnnData accumlated the following annotations.


In [ ]:
adata

## Inspirations

- [Analysis of single cell RNA-seq data](https://www.singlecellcourse.org/)
    - Course from University of Cambridge Bioinformatics training unit
    - In R
- [Single cell study database](www.nxn.se/single-cell-studies/)
- [Scanpy tutorials](https://scanpy-tutorials.readthedocs.io/en/latest/)